In [4]:
import os
# from utils.Preprocessing import filter,features,plots
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

This code uploads all files from a country

In [38]:
# file_path = "Organized Data/china/"
# file_list = os.listdir(file_path)
# print(file_list)
# df = pd.DataFrame()

# for file in file_list:
#     # if 'abb_china' in file and file.endswith(".csv"):
#     print(file)
#     df_temp = pd.read_csv(os.path.join(file_path, file), index_col=0)
#     df = pd.concat([df, df_temp])

# # dataset=dataset.sample(frac=1).reset_index(drop=True)
# df.fillna(0, inplace=True)
# df

This code is used to upload data from a single file

In [13]:
df=pd.read_csv('Organized Data/china/abb_china_36435_statistics.csv')
# Fill NaN values with zero
df.fillna(0, inplace=True)
df

,Unnamed: 0,TFF,CT,TAV,SHOT_COUNT,STATISTICS_ID,MOLD_ID,MOLD_CODE,COUNTER_ID,CDATA_ID,SC,WACT,ACT,TOOLING_TYPE,TOTAL_CAVITIES,max_shots,DATE,HOUR,LIFE_CYCLE,PHASE
0,0,20200922142915,0.0,265,40,40014,36435,2TAP200001R0010,NCM2029I01030,40012,40,144.554702,150,Injection Mold,4,1500000,20200922,2020092214,0.000027,1
1,1,20200922152917,0.0,265,0,40204,36435,2TAP200001R0010,NCM2029I01030,40202,40,144.554702,150,Injection Mold,4,1500000,20200922,2020092215,0.000027,1
2,2,20200922162919,0.0,267,0,40366,36435,2TAP200001R0010,NCM2029I01030,40364,40,144.554702,150,Injection Mold,4,1500000,20200922,2020092216,0.000027,1
3,3,20200922172921,0.0,268,0,40529,36435,2TAP200001R0010,NCM2029I01030,40527,40,144.554702,150,Injection Mold,4,1500000,20200922,2020092217,0.000027,1
4,4,20200922182923,0.0,269,0,40691,36435,2TAP200001R0010,NCM2029I01030,40689,40,144.554702,150,Injection Mold,4,1500000,20200922,2020092218,0.000027,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18315,18317,20231031100204,0.0,221,0,3265659,36435,2TAP200001R0010,NCM2029I01030,3265657,1562788,144.554702,150,Injection Mold,4,1500000,20231031,2023103110,1.041859,4
18316,18318,20231031110205,0.0,223,0,3265665,36435,2TAP200001R0010,NCM2029I01030,3265663,1562788,144.554702,150,Injection Mold,4,1500000,20231031,2023103111,1.041859,4
18317,18319,20231031120206,0.0,221,0,3265674,36435,2TAP200001R0010,NCM2029I01030,3265670,1562788,144.554702,150,Injection Mold,4,1500000,20231031,2023103112,1.041859,4
18318,18320,20231031130207,0.0,222,0,3265679,36435,2TAP200001R0010,NCM2029I01030,3265677,1562788,144.554702,150,Injection Mold,4,1500000,20231031,2023103113,1.041859,4


In [14]:
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
df

,HOUR,CT,TAV,SHOT_COUNT,PHASE
0,2020-09-22 14:00:00,0.0,265,40,1
1,2020-09-22 15:00:00,0.0,265,0,1
2,2020-09-22 16:00:00,0.0,267,0,1
3,2020-09-22 17:00:00,0.0,268,0,1
4,2020-09-22 18:00:00,0.0,269,0,1
...,...,...,...,...,...
18315,2023-10-31 10:00:00,0.0,221,0,4
18316,2023-10-31 11:00:00,0.0,223,0,4
18317,2023-10-31 12:00:00,0.0,221,0,4
18318,2023-10-31 13:00:00,0.0,222,0,4


In [17]:
# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")
y_train

Training set shape: (14080, 720, 3), (14080, 720, 4)
Testing set shape: (3520, 720, 3), (3520, 720, 4)


array([[[0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        ...,
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0]],

       [[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        ...,
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]],

       [[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        ...,
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]],

       ...,

       [[0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        ...,
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]],

       [[1, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        ...,
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0]],

       [[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        ...,
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1]]], dtype=uint8)

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import TimeDistributed

# Assuming X_train, y_train, X_test, y_test are your training and testing sets
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[2]

# Function to create the model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# # Function to fine-tune hyperparameters
# def fine_tune_model(X_train, y_train):
#     model = KerasClassifier(build_fn=create_model, verbose=0)
#     # define the grid search parameters
#     optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#     param_grid = dict(optimizer=optimizer)
#     grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
#     grid_result = grid.fit(X_train, y_train)
#     # summarize results
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#     means = grid_result.cv_results_['mean_test_score']
#     stds = grid_result.cv_results_['std_test_score']
#     params = grid_result.cv_results_['params']
#     for mean, stdev, param in zip(means, stds, params):
#         print("%f (%f) with: %r" % (mean, stdev, param))

# Create the LSTM model
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=35, batch_size=32, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Fine-tune hyperparameters
# fine_tune_model(X_train, y_train)


Epoch 1/35
440/440 - 86s - loss: 0.9719 - accuracy: 0.5899
Epoch 2/35
440/440 - 84s - loss: 0.6686 - accuracy: 0.7486
Epoch 3/35
440/440 - 85s - loss: 0.5195 - accuracy: 0.8053
Epoch 4/35
440/440 - 83s - loss: 0.4714 - accuracy: 0.8178
Epoch 5/35
440/440 - 83s - loss: 0.4505 - accuracy: 0.8256
Epoch 6/35
440/440 - 83s - loss: 0.4256 - accuracy: 0.8319
Epoch 7/35
440/440 - 99s - loss: 0.3947 - accuracy: 0.8435
Epoch 8/35
440/440 - 82s - loss: 0.4115 - accuracy: 0.8407
Epoch 9/35
440/440 - 84s - loss: 0.3774 - accuracy: 0.8524
Epoch 10/35
440/440 - 84s - loss: 0.3584 - accuracy: 0.8630
Epoch 11/35
440/440 - 86s - loss: 0.3474 - accuracy: 0.8677
Epoch 12/35
440/440 - 84s - loss: 0.3642 - accuracy: 0.8607
Epoch 13/35
440/440 - 83s - loss: 0.3413 - accuracy: 0.8695
Epoch 14/35
440/440 - 93s - loss: 0.3400 - accuracy: 0.8684
Epoch 15/35
440/440 - 88s - loss: 0.3252 - accuracy: 0.8735
Epoch 16/35
440/440 - 88s - loss: 0.3159 - accuracy: 0.8745
Epoch 17/35
440/440 - 103s - loss: 0.3214 - accur

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Bidirectional, LSTM

# # Define the BiLSTM model
# model = Sequential()
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(time_steps, n_features)))
# model.add(Dense(4, activation='softmax'))  # Assuming there are 4 phases

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X, y, epochs=10, verbose=1)

# # Evaluate the model
# loss, accuracy = model.evaluate(X, y)
# print('Loss: %.2f' % (loss*100))
# print('Accuracy: %.2f' % (accuracy*100))


In [19]:
model.save('Weights/abb_china_36435.h5')

In [20]:
from keras.models import load_model
model=load_model('Weights/abb_china_36435.h5')

Testing on other dataset

In [21]:
df = pd.read_csv('Organized Data/china/abb_china_36505_statistics.csv')
df.fillna(0, inplace=True)
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

loss, accuracy = model.evaluate(X, y, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy*100}")

457/457 - 27s - loss: 3.5842 - accuracy: 0.2909
Loss: 3.584200382232666, Accuracy: 29.09359037876129


Test for all files in a folder

In [22]:
# Folder containing CSV files
folder_path = 'Organized Data/china/'

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Construct the full path to the CSV file
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file
        df = pd.read_csv(file_path)
        df.fillna(0, inplace=True)
        df = df[['HOUR', 'CT', 'TAV', 'SHOT_COUNT', 'PHASE']]

        # Assuming df is your DataFrame and 'HOUR' is your time column
        df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
        df = df.sort_values('HOUR')
        features_considered = ['CT', 'TAV', 'SHOT_COUNT']

        # One-hot encoding the 'PHASE' feature
        y = pd.get_dummies(df['PHASE']).values

        # Creating the sliding window
        def create_dataset(X, y, time_steps=1):
            Xs, ys = [], []
            for i in range(len(X) - time_steps):
                v = X.iloc[i:(i + time_steps)].values
                Xs.append(v)
                ys.append(y[i:i + time_steps])
            return np.array(Xs), np.array(ys)

        time_steps = 720  # Represents 30 days of hourly data

        # Reshape to [samples, time_steps, n_features]
        X, y = create_dataset(df[features_considered], y, time_steps)

        # Evaluate the model on the data
        loss, accuracy = model.evaluate(X, y, verbose=2)

        # Print filename, loss, and accuracy
        print(f"File: {filename}, Loss: {loss}, Accuracy: {np.round(accuracy * 100,2)}")

537/537 - 35s - loss: 4.9627 - accuracy: 0.2913
File: abb_china_36405_statistics.csv, Loss: 4.962714195251465, Accuracy: 29.13
550/550 - 38s - loss: 0.2651 - accuracy: 0.8912
File: abb_china_36435_statistics.csv, Loss: 0.2651156485080719, Accuracy: 89.12
475/475 - 33s - loss: 5.5476 - accuracy: 0.3299
File: abb_china_36440_statistics.csv, Loss: 5.547614097595215, Accuracy: 32.99
464/464 - 35s - loss: 3.9074 - accuracy: 0.3522
File: abb_china_36445_statistics.csv, Loss: 3.9073894023895264, Accuracy: 35.22
489/489 - 37s - loss: 5.4034 - accuracy: 0.2484
File: abb_china_36450_statistics.csv, Loss: 5.403362274169922, Accuracy: 24.84
490/490 - 33s - loss: 3.2003 - accuracy: 0.4358
File: abb_china_36455_statistics.csv, Loss: 3.2002925872802734, Accuracy: 43.58
508/508 - 36s - loss: 5.5715 - accuracy: 0.3065
File: abb_china_36460_statistics.csv, Loss: 5.571467399597168, Accuracy: 30.65
549/549 - 41s - loss: 3.4802 - accuracy: 0.3262
File: abb_china_36475_statistics.csv, Loss: 3.48024606704711